In [11]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from load_snowflake import load_snowflake
from urllib.parse import quote_plus 
load_dotenv()

api_key = os.getenv('API_KEY')

In [12]:
def fetch_events():
    # Your Ticketmaster API key
    api_key = os.getenv('API_KEY')
    
    # Base URL for Ticketmaster API
    url = 'https://app.ticketmaster.com/discovery/v2/events'
    
    # Initial API parameters
    params = {
        'apikey': api_key,
        'countryCode': 'US',                      # Limit results to the US
        'startDateTime': '2024-12-01T00:00:00Z',  # Start date
        ### Deep paging issues over 1000 items, limiting to 1 month for now. will need to add batching
        'endDateTime': '2024-12-31T23:59:59Z',    # End date
        'size': 200,                              # Max events per page
        'page': 0,                                # Start at page 0
        'sort': 'date,asc'                        # Sort by date ascending
    }
    
    all_events = []  # To store all events data
    
    while True:
        # Fetch the events for the current page
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        if response.status_code != 200:
            print(f"Error: Unable to fetch data (status code {response.status_code})")
            break
        
        # Get the JSON response
        data = response.json()

        # Check if there are any events in the response
        if '_embedded' not in data or 'events' not in data['_embedded']:
            print("No more events found.")
            break
        
        # Extract events from the current page
        events = data['_embedded']['events']
        all_events.extend(events)
        
        # Print the number of events fetched on this page
        print(f"Fetched {len(events)} events on page {params['page']}")
        
        # Check if there are more pages
        if params['page'] >= data['page']['totalPages'] - 1:
            print("All pages have been fetched.")
            break
        
        # Increment to the next page
        params['page'] += 1
    
    
    return all_events

In [13]:
def events_to_dataframe(events):
    # Create a list to store processed event data
    event_list = []

    for event in events:
        # Extract relevant fields from each event
        event_name = event.get('name')
        event_date = event['dates']['start'].get('localDate')
        event_time = event['dates']['start'].get('localTime', 'N/A')  
        venue_name = event['_embedded']['venues'][0].get('name')
    #    venue_city = event['_embedded']['venues'][0]['city'].get('name', 'N/A')
     #   venue_state = event['_embedded']['venues'][0]['state'].get('name', 'N/A')  
        
        # Append event data to the list
        event_list.append([event_name, event_date, event_time, venue_name #, venue_city, venue_state
                           ])

    # Convert the list to a DataFrame
    df = pd.DataFrame(event_list, columns=['Event Name', 'Date', 'Time', 'Venue' #, 'City', 'State'
                                           ])
    return df

In [20]:
def events_to_dataframe_allCol(events):
    df = pd.json_normalize(events, sep='_')

    return df


In [21]:
events_data = fetch_events()
events_df = events_to_dataframe_allCol(events_data)


Fetched 200 events on page 0
Fetched 200 events on page 1
Fetched 200 events on page 2
Fetched 200 events on page 3
Fetched 200 events on page 4
Error: Unable to fetch data (status code 400)


In [25]:
print(events_df.columns)

Index(['name', 'type', 'id', 'test', 'url', 'locale', 'images',
       'classifications', 'info', 'pleaseNote', 'priceRanges',
       'sales_public_startDateTime', 'sales_public_startTBD',
       'sales_public_startTBA', 'sales_public_endDateTime',
       'dates_access_startDateTime', 'dates_access_startApproximate',
       'dates_access_endDateTime', 'dates_access_endApproximate',
       'dates_start_localDate', 'dates_start_localTime',
       'dates_start_dateTime', 'dates_start_dateTBD', 'dates_start_dateTBA',
       'dates_start_timeTBA', 'dates_start_noSpecificTime',
       'dates_end_localDate', 'dates_end_localTime', 'dates_end_dateTime',
       'dates_end_approximate', 'dates_end_noSpecificTime', 'dates_timezone',
       'dates_status_code', 'dates_spanMultipleDays', '_links_self_href',
       '_links_venues', '_embedded_venues', '_links_attractions',
       '_embedded_attractions', 'ticketing_safeTix_enabled',
       'ticketing_safeTix_inAppOnlyEnabled', 'ticketing_id', 'descr

In [15]:
events_data[55]

{'name': 'PAW PATROL LIVE! 11/30 2PM - CLUB',
 'type': 'event',
 'id': 'Za5ju3rKuqZB48J4CyIWe15opz4QSvXYq',
 'test': False,
 'locale': 'en-us',
 'images': [{'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7899-4f67-8067-1f95f4d158cf_124761_RETINA_LANDSCAPE_16_9.jpg',
   'width': 1136,
   'height': 639,
   'fallback': True},
  {'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7899-4f67-8067-1f95f4d158cf_124761_RETINA_PORTRAIT_16_9.jpg',
   'width': 640,
   'height': 360,
   'fallback': True},
  {'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7899-4f67-8067-1f95f4d158cf_124761_RECOMENDATION_16_9.jpg',
   'width': 100,
   'height': 56,
   'fallback': True},
  {'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7899-4f67-8067-1f95f4d158cf_124761_TABLET_LANDSCAPE_LARGE_16_9.jpg',
   'width': 2048,
   'height': 1152,
   'fallback': True},
  {'ratio': '3_2',
   'url': 'https://s1.ticketm.net/dam/c/8cf/a665388

In [16]:
def event_one(id):
    url = f'https://app.ticketmaster.com/discovery/v2/events/{id}'
    
    params = {
        'apikey': api_key,
        'countryCode': 'US',                      # Limit results to the US
        'startDateTime': '2024-12-01T00:00:00Z',  # Start date
        ### Deep paging issues over 1000 items, limiting to 1 month for now. will need to add batching
        'endDateTime': '2024-12-31T23:59:59Z',    # End date
        'size': 200,                              # Max events per page
        'page': 0,                                # Start at page 0
        'sort': 'date,asc'                        # Sort by date ascending
    }
    
    all_events = []  # To store all events data
    
    # Fetch the events for the current page
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code != 200:
        print(f"Error: Unable to fetch data (status code {response.status_code})")
        
    # Get the JSON response
    data = response.json()
    return data

In [17]:
event_one('Za5ju3rKuqZBYltJm4lPeb5htjfKJELn8')

{'name': 'How The Grinch Stole Christmas',
 'type': 'event',
 'id': 'Za5ju3rKuqZBYltJm4lPeb5htjfKJELn8',
 'test': False,
 'locale': 'en-us',
 'images': [{'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7899-4f67-8067-1f95f4d158cf_124761_RETINA_LANDSCAPE_16_9.jpg',
   'width': 1136,
   'height': 639,
   'fallback': True},
  {'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7899-4f67-8067-1f95f4d158cf_124761_RETINA_PORTRAIT_16_9.jpg',
   'width': 640,
   'height': 360,
   'fallback': True},
  {'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7899-4f67-8067-1f95f4d158cf_124761_RECOMENDATION_16_9.jpg',
   'width': 100,
   'height': 56,
   'fallback': True},
  {'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7899-4f67-8067-1f95f4d158cf_124761_TABLET_LANDSCAPE_LARGE_16_9.jpg',
   'width': 2048,
   'height': 1152,
   'fallback': True},
  {'ratio': '3_2',
   'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7